# Defining the Statement of Problem

<li> This notebook implements a notebook recommender system
<li> Recommender systems are used to suggest movies or songs to users based on their interest or usage history
<li> In this example we will use User and Item-bases Collaborative Filter
<li> Dataset MovieLens: <a href="https://grouplens.org/datasets/movielens/100k/">https://grouplens.org/datasets/movielens/100k/</a>
<li> Photo Credit: <a href="https://pxhere.com/en/photo/1588369">https://pxhere.com/en/photo/1588369</a>
